In [2]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np


In [3]:
AlleFluegeURL = "https://gist.githubusercontent.com/florianeichin/cfa1705e12ebd75ff4c321427126ccee/raw/c86301a0e5d0c1757d325424b8deec04cc5c5ca9/flights_all_cleaned.csv "
SubsetURL = "https://gist.githubusercontent.com/florianeichin/b877d354d6bc52e6ce840572e40b0497/raw/19759410471073756a388dada5fcb40109f0d13e/flights_subset_cleaned.csv"
OriginalDatenURL = "https://www.kaggle.com/usdot/flight-delays#flights.csv"
localeUncleanedCSV = "flights.csv"

# Loading Data from the URL
df = pd.read_csv(localeUncleanedCSV)
pd.set_option("display.max_columns", None)

C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
# dropping all airports with unknown (not in airports.csv) IATA_CODE (df.ORIGIN_AIRPOPRT)
print(len(df))

dropL = []
for i, row in df.iterrows():
    if row.ORIGIN_AIRPORT == type(int):
        dropL.append(i)

df = df.drop(df.index(dropL))

print(len(df))

5819079


TypeError: 'RangeIndex' object is not callable

In [4]:
# dropping rows with many NaN values and irrelevant columns
df = df.drop(["FLIGHT_NUMBER", "TAIL_NUMBER", "TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "TAXI_IN", "CANCELLATION_REASON", "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY","LATE_AIRCRAFT_DELAY", "WEATHER_DELAY", "DIVERTED"], axis = 1)
df["SCHEDULED_DEPARTURE"] = df["SCHEDULED_DEPARTURE"].astype("string")
df["DEPARTURE_TIME"] = df["DEPARTURE_TIME"].astype("string")
df["SCHEDULED_DEPARTURE"].dropna()
df["DEPARTURE_TIME"].dropna()
# df.dropna()

0          2354.0
1             2.0
2            18.0
3            15.0
4            24.0
            ...  
5819074    2355.0
5819075    2355.0
5819076    2350.0
5819077    2353.0
5819078      14.0
Name: DEPARTURE_TIME, Length: 5732926, dtype: string

In [5]:
# reformatting the "SCHEDULED_DEPARTURE" column to "xx:xx"


for i, row in df.iterrows():
    if len(row["SCHEDULED_DEPARTURE"]) == 1:
        df["SCHEDULED_DEPARTURE"][i] = "00:0" + row["SCHEDULED_DEPARTURE"] 
    elif len(row["SCHEDULED_DEPARTURE"]) == 2:
        df["SCHEDULED_DEPARTURE"][i] = "00:" + row["SCHEDULED_DEPARTURE"] 
    elif len(row["SCHEDULED_DEPARTURE"]) == 3:
        df["SCHEDULED_DEPARTURE"][i] = "0" + row["SCHEDULED_DEPARTURE"][:1] + ":" + row["SCHEDULED_DEPARTURE"][1:]
    elif len(row["SCHEDULED_DEPARTURE"]) == 4:
        df["SCHEDULED_DEPARTURE"][i] = row["SCHEDULED_DEPARTURE"][:2] + ":" + row["SCHEDULED_DEPARTURE"][2:]


# reformatting the "DEPARTURE_TIME" column to "xx:xx"
for i, row in df.iterrows():
    if pd.isnull(row["DEPARTURE_TIME"]):
        continue
    elif len(row["DEPARTURE_TIME"]) == 3:
        df["DEPARTURE_TIME"][i] = "00:0" + row["DEPARTURE_TIME"] 
    elif len(row["DEPARTURE_TIME"]) == 4:
        df["DEPARTURE_TIME"][i] = "00:" + row["DEPARTURE_TIME"] 
    elif len(row["DEPARTURE_TIME"]) == 5:
        df["DEPARTURE_TIME"][i] = "0" + row["DEPARTURE_TIME"][:1] + ":" + row["DEPARTURE_TIME"][1:]
    elif len(row["DEPARTURE_TIME"]) == 6:
        df["DEPARTURE_TIME"][i] = row["DEPARTURE_TIME"][:2] + ":" + row["DEPARTURE_TIME"][2:]




In [6]:
# adding zeros before single digit months and days

indicesOfDays = []
indicesOfMonths = []

for i, row in df.iterrows():
    if row["DAY"] < 10:
        indicesOfDays.append(i)
    if row["MONTH"] < 10:
        indicesOfMonths.append(i)

df["DAY"] = df["DAY"].astype("string")
df["MONTH"] = df["MONTH"].astype("string")

for i in indicesOfDays:
    df["DAY"][i] = "0" + df["DAY"][i]


for i in indicesOfMonths:
    df["MONTH"][i] = "0" + df["MONTH"][i]



In [7]:
# adding new column with year, month and day
df["YEAR"] = df["YEAR"].astype("string")
new_col = df["YEAR"] + "-" + df["MONTH"] + "-" + df["DAY"]
df.insert(0, "FULL_DATE", new_col)

In [8]:
df.tail()

,FULL_DATE,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,CANCELLED
5819074,2015-12-31,2015,12,31,4,B6,LAX,BOS,23:59,23:55.0,-4.0,320.0,298.0,272.0,2611,819,753.0,-26.0,0
5819075,2015-12-31,2015,12,31,4,B6,JFK,PSE,23:59,23:55.0,-4.0,227.0,215.0,195.0,1617,446,430.0,-16.0,0
5819076,2015-12-31,2015,12,31,4,B6,JFK,SJU,23:59,23:50.0,-9.0,221.0,222.0,197.0,1598,440,432.0,-8.0,0
5819077,2015-12-31,2015,12,31,4,B6,MCO,SJU,23:59,23:53.0,-6.0,161.0,157.0,144.0,1189,340,330.0,-10.0,0
5819078,2015-12-31,2015,12,31,4,B6,JFK,BQN,23:59,00:14.0,15.0,221.0,208.0,189.0,1576,440,442.0,2.0,0


In [9]:
# replacing weekday int by string of weekday
df["DAY_OF_WEEK"][df["DAY_OF_WEEK"] == 1] = "Monday"
df["DAY_OF_WEEK"][df["DAY_OF_WEEK"] == 2] = "Tuesday"
df["DAY_OF_WEEK"][df["DAY_OF_WEEK"] == 3] = "Wednessday"
df["DAY_OF_WEEK"][df["DAY_OF_WEEK"] == 4] = "Thursday"
df["DAY_OF_WEEK"][df["DAY_OF_WEEK"] == 5] = "Friday"
df["DAY_OF_WEEK"][df["DAY_OF_WEEK"] == 6] = "Saturday"
df["DAY_OF_WEEK"][df["DAY_OF_WEEK"] == 7] = "Sunday"


C:\Users\Florian\AppData\Local\Temp\ipykernel_2184\281417417.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DAY_OF_WEEK"][df["DAY_OF_WEEK"] == 1] = "Monday"
C:\Users\Florian\AppData\Local\Temp\ipykernel_2184\281417417.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DAY_OF_WEEK"][df["DAY_OF_WEEK"] == 2] = "Tuesday"
C:\Users\Florian\AppData\Local\Temp\ipykernel_2184\281417417.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [10]:
df.to_csv("CleanedFlights.csv")